In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [1]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
from datetime import datetime
import pandas as pd

# Fonction pour extraire les informations d'un produit à partir de son lien
def scrape_product_info(product_link):
    response = requests.get(product_link)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        product_name_element = soup.find('h1', {'itemprop': 'name'})
        product_name = product_name_element.text.strip() if product_name_element else None

        image_element = soup.find('img', {'class': 'js-qv-product-cover'})
        image_link = image_element.get('src') if image_element else None

        product_price_element = soup.find('span', {'itemprop': 'price'})
        product_price_raw = product_price_element.text.strip() if product_price_element else None

        # Convertir la chaîne de prix en un nombre décimal avec trois chiffres après la virgule
        try:
            product_price = round(float(product_price_raw.replace("TND", "").replace("TTC", "").replace(",", ".").strip()), 3)
        except ValueError:
            print(f"Impossible de convertir le prix en un nombre décimal: {product_price_raw}")
            return None

        return {
            'name': product_name,
            'nom_site': 'Pharma Shop',
            'image_link': image_link,
            'price': product_price,
            'product_link': product_link
        }
    else:
        print(f"La requête a échoué avec le code de statut : {response.status_code}")
        return None

# Fonction pour extraire les informations de tous les produits d'une page
def scrape_all_products(url):
    page_number = 1

    # Créer une liste de dictionnaires pour stocker les données des produits
    products_data = []

    while True:
        current_url = f"{url}?page={page_number}"
        response = requests.get(current_url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            product_links = soup.select('.thumbnail.product-thumbnail')

            if not product_links:
                break

            for link in product_links:
                product_link = link.get('href')
                product_info = scrape_product_info(product_link)

                if product_info:
                    products_data.append(product_info)
                    print("Lien du produit:", product_info['product_link'])
                    print("Nom du produit:", product_info['name'])
                    print("Nom du site:", product_info['nom_site'])
                    print("Lien de l'image:", product_info['image_link'])
                    print("Prix du produit:", product_info['price'])
                    print("-" * 50)

            page_number += 1
        else:
            print(f"La requête a échoué avec le code de statut : {response.status_code}")
            break

    # Créer un DataFrame à partir des données des produits
    df = pd.DataFrame(products_data)

    # Supprimer les duplicatas basés sur le lien du produit
    df.drop_duplicates(subset=['product_link'], keep='first', inplace=True)

    # Appeler la fonction pour insérer les données dans la base de données
    insert_into_database(df)

# Fonction pour insérer les données dans la base de données MySQL
def insert_into_database(df):
    db_config = {
        'host': 'localhost',
        'user': 'root',
        'database': 'scrapping_db',
        'port': 3306
    }

    connexion = mysql.connector.connect(**db_config)
    curseur = connexion.cursor()

    try:
        for index, row in df.iterrows():
            curseur.execute("""
                INSERT INTO produits (nom, nom_site, image, lien)
                VALUES (%s, %s, %s, %s)
            """, (row['name'], row['nom_site'], row['image_link'], row['product_link']))

            current_date = datetime.today().strftime('%Y-%m-%d')

            curseur.execute("""
                INSERT INTO prix (id_produit, prix, date_prix)
                VALUES (LAST_INSERT_ID(), %s, %s)
            """, (row['price'], current_date))

        connexion.commit()
        print("Données insérées dans la base de données MySQL.")
    except Exception as e:
        print(f"Une erreur s'est produite lors de l'insertion dans la base de données : {e}")
    finally:
        curseur.close()
        connexion.close()

# URL de la page contenant la liste des produits
base_url = "https://pharma-shop.tn/839-visage"

# Appeler la fonction pour extraire les informations de tous les produits de la première page
scrape_all_products(base_url)


Lien du produit: https://pharma-shop.tn/eau-tonique-lotion/4411-dermedic-hydrain-3-eau-micellaire-500-ml-5901643174569.html
Nom du produit: DERMEDIC HYDRAIN 3 EAU MICELLAIRE 500 ML
Nom du site: Pharma Shop
Lien de l'image: https://pharma-shop.tn/6482-large_default/dermedic-hydrain-3-eau-micellaire-500-ml.webp
Prix du produit: 20.5
--------------------------------------------------
Lien du produit: https://pharma-shop.tn/lait-mousse-emulsion/1483-pharmaceris-w-purialbucin-mousse-5900717147010.html
Nom du produit: PHARMACERIS W PURI-ALBUCIN MOUSSE NETTOYANTE ECLAIRCISSANTE, 150ML
Nom du site: Pharma Shop
Lien de l'image: https://pharma-shop.tn/3764-large_default/pharmaceris-w-purialbucin-mousse.webp
Prix du produit: 26.835
--------------------------------------------------


KeyboardInterrupt: 